# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [ ]:
!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

In [1]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.2.0'

## Get Required Files from Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#**## Reading and Exploring Data**

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [4]:
#Set your project path 
project_path =  ("/content/drive/My Drive/LSTM/Data/")

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [5]:
df = pd.read_json(project_path+'Sarcasm_Headlines_Dataset.json', lines=True)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [6]:
df.isna().sum() # Checking for NaN values

article_link    0
headline        0
is_sarcastic    0
dtype: int64

## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [7]:
del df['article_link'] # Deleting this column as it is of no use

In [8]:
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [9]:
line_length = []
for i in range(len(df["headline"])):
  List = (df["headline"].iloc[i].split())
  line_length.append(len(List))
print(line_length)

[12, 14, 14, 13, 11, 4, 7, 14, 7, 9, 10, 8, 12, 10, 8, 8, 16, 7, 9, 3, 7, 16, 11, 9, 13, 7, 8, 4, 7, 7, 8, 10, 12, 9, 8, 8, 5, 15, 11, 10, 10, 15, 8, 8, 8, 6, 15, 11, 7, 4, 4, 11, 9, 13, 14, 11, 11, 9, 10, 12, 4, 5, 6, 13, 12, 9, 12, 17, 7, 7, 8, 9, 9, 8, 13, 11, 9, 10, 5, 8, 10, 8, 10, 13, 10, 11, 14, 11, 17, 13, 4, 19, 12, 8, 11, 9, 10, 10, 11, 9, 7, 12, 9, 10, 6, 9, 7, 10, 6, 11, 12, 12, 7, 22, 9, 11, 10, 19, 12, 13, 8, 3, 10, 10, 9, 9, 11, 8, 4, 14, 11, 8, 6, 11, 8, 7, 11, 9, 11, 9, 11, 11, 8, 6, 10, 8, 10, 9, 11, 15, 9, 8, 6, 11, 7, 9, 8, 11, 7, 18, 9, 7, 9, 14, 14, 13, 9, 15, 11, 8, 8, 10, 14, 9, 9, 6, 12, 5, 10, 7, 13, 14, 8, 7, 10, 9, 15, 10, 11, 11, 11, 12, 6, 8, 9, 12, 6, 8, 12, 16, 7, 14, 12, 9, 7, 13, 6, 7, 12, 9, 10, 6, 13, 15, 6, 22, 9, 11, 15, 9, 12, 10, 12, 10, 5, 8, 10, 9, 12, 5, 9, 8, 10, 11, 7, 9, 12, 12, 9, 8, 6, 14, 10, 11, 6, 7, 5, 13, 5, 8, 14, 9, 8, 9, 4, 5, 17, 13, 10, 7, 10, 12, 11, 8, 4, 7, 8, 10, 7, 11, 12, 12, 8, 16, 7, 8, 6, 8, 4, 9, 8, 6, 14, 8, 11, 14, 8

In [10]:
print("Maximum Length of sentense is ", max(line_length))

Maximum Length of sentense is  39


#**## Modelling**

## Import required modules required for modelling.

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [12]:
max_features = 10000
maxlen = 39
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [13]:
# Pre processing - Tokenization
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df["headline"])
#tokenized_train = tokenizer.texts_to_sequences(x_train)
#x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

# Define X and y for your model.

In [14]:
X = tokenizer.texts_to_sequences(df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(df['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0  307  678 3336 2297   47  381 2575    5 2576 8433]
Number of Labels:  26709
0


## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [15]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 29656 unique tokens.


#**## Word Embedding**

## Get Glove Word Embeddings

In [16]:
glove_file = project_path + "glove.6B.zip"

In [17]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [20]:
EMBEDDING_FILE = project_path +'/glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd



In [21]:
embeddings

Output hidden; open in https://colab.research.google.com to view.

# Create a weight matrix for words in training docs

In [22]:
num_words=len(word_index)+1
embedding_matrix = np.zeros((num_words, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [23]:
#Defining Neural Network
tf.keras.backend.clear_session()
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(num_words,embedding_size, weights=[embedding_matrix], input_length=200, trainable=True))
#LSTM 
model.add(Bidirectional(LSTM(units=128 , recurrent_dropout = 0.5 , dropout = 0.5)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 200)          5931400   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               336896    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 6,268,553
Trainable params: 6,268,553
Non-trainable params: 0
_________________________________________________________________


# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [25]:
batch_size = 100
epochs = 5

model.fit(X, y, epochs=5, batch_size=100, validation_split=0.2)
## Add your code here ##
#history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (X_test,y_test) , epochs = 2)

Epoch 1/5
214/214 [==============================] - 75s 352ms/step - loss: 0.4812 - accuracy: 0.7619 - val_loss: 0.3627 - val_accuracy: 0.8424
Epoch 2/5
214/214 [==============================] - 75s 349ms/step - loss: 0.3163 - accuracy: 0.8646 - val_loss: 0.3190 - val_accuracy: 0.8615
Epoch 3/5
214/214 [==============================] - 74s 345ms/step - loss: 0.2493 - accuracy: 0.8957 - val_loss: 0.3150 - val_accuracy: 0.8688
Epoch 4/5
214/214 [==============================] - 74s 346ms/step - loss: 0.1960 - accuracy: 0.9221 - val_loss: 0.3498 - val_accuracy: 0.8675
Epoch 5/5
214/214 [==============================] - 73s 342ms/step - loss: 0.1639 - accuracy: 0.9356 - val_loss: 0.3599 - val_accuracy: 0.8639


In [26]:
x_train,x_test,y_train,y_test = train_test_split(df.headline,df.is_sarcastic, test_size = 0.3 , random_state = 0)

In [27]:
tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [28]:
print("Accuracy of the model on Training Data is - " , model.evaluate(X,y)[1]*100)

835/835 [==============================] - 17s 20ms/step - loss: 0.1468 - accuracy: 0.9472
Accuracy of the model on Training Data is -  94.7246253490448


In [29]:
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test,y_test)[1]*100)

251/251 [==============================] - 5s 20ms/step - loss: 0.1508 - accuracy: 0.9441
Accuracy of the model on Testing Data is -  94.40908432006836


In [30]:
pred = model.predict_classes(X_test)
pred[:5]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[1],
       [1],
       [1],
       [0],
       [1]], dtype=int32)

In [31]:
print(classification_report(y_test, pred, target_names = ['Not Sarcastic','Sarcastic']))

               precision    recall  f1-score   support

Not Sarcastic       0.96      0.94      0.95      4483
    Sarcastic       0.93      0.95      0.94      3530

     accuracy                           0.94      8013
    macro avg       0.94      0.94      0.94      8013
 weighted avg       0.94      0.94      0.94      8013



In [32]:
cm = confusion_matrix(y_test,pred)
cm

array([[4220,  263],
       [ 185, 3345]])